# Test 1: gforecast Branch, g Forecast

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill

# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
datastore = cf.correct_path("/Volumes/SSD01/data.nosynctest1/")
classpath = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/class")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps18_resultsdir = cf.correct_path("/Volumes/SSD01/results/test1/")
cf.makedirectory(ps18_resultsdir)

# Linda Fiducial Cosmology 
ps18_fid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.967,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.1127,
        "tau_reio" : 0.0598,
        "h" : 0.701,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "deg_ncdm" : 1,
        "T_ncdm" : (0.91/2.726), # Units [T_cmb]. 
        "m_ncdm" : 0.01, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000, #Units [m s^-2]
        "N_eff" : 3.046, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm" # Fix T_ncdm or m_ncdm 
        }

# DESI Parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Fisher Forecast
masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.91])

omegacdm_set = np.array([ps18_fid['omega_cdm'] - ((masses/cf.NEUTRINO_SCALE_FACTOR)*np.power(tval / 1.95, 3.)) for tidx, tval in enumerate(temps)])                                  
ps18_fiducialset = [[dict(ps18_fid, **{
    'm_ncdm' : masses[midx], 
    'omega_cdm' : omegacdm_set[tidx, midx], 
    'T_ncdm' : temps[tidx]/2.726}) 
    for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)] 
ps18_forecastset = [[cf.forecast(
    classpath, 
    datastore, 
    'relic', 
    fidval, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True) 
    for fididx, fidval in enumerate(fidrowvals)] 
    for fidrowidx, fidrowvals in enumerate(ps18_fiducialset)]

for frowidx, frowval in enumerate(ps18_forecastset):
    for fidx, fcst in enumerate(frowval):
        if type(fcst.fisher)==type(None):
            fcst.gen_pm()
            fcst.gen_fisher(
                fisher_order=[
                    'omega_b',
                    'omega_cdm',
                    'n_s',
                    'A_s',
                    'tau_reio',
                    'h',
                    'N_ncdm',
                    'sigma_fog',
                    'b0',
                    'alpha_k2'],
                mu_step=mu_integral_step,
                skipgen=False)
            print("Relic Forecast ", fidx, " complete...")
            dill.dump_session(os.path.join(ps18_resultsdir, 'test1.db'))
        else:
            print('Fisher matrix already generated!')

In [ ]:
import dill
dill.load_session('/Volumes/SSD01/results/test1/test1.db')

ps18_resultsdir = cf.correct_path("/Volumes/SSD01/results/test1/")

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames=['0.91']

head = 'A_s \t h \t n_s \t omega_b \t omega_cdm \t tau_reio \t N_ncdm \t sigma_fog \t beta0 \t alpha_k2'


for tidx, tval in enumerate(ps18_forecastset):
    for midx, mval in enumerate(tval): 
        mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
              fisher_order=[
                'A_s',
                'h',  
                'n_s',
                'omega_b',                                    
                'omega_cdm',                                                     
                'tau_reio',                                                                                    
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/Relic_Fine_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps18_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)



# Test 2: gforecast Branch, M Forecast

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns 
import cosmicfish as cf 
import matplotlib.pyplot as plt
import dill

# Instruct pyplot to use seaborn 
sns.set()

# Set project, data, CLASS directories 
datastore = cf.correct_path("/Volumes/SSD01/data.nosynctest2/")
classpath = cf.correct_path("/Users/nicholasdeporzio/Desktop/cfworkspace/class2")

# Specify resolution of numerical integrals
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths  
ps2_resultsdir = cf.correct_path('/Volumes/SSD01/results/test2/')
cf.makedirectory(ps2_resultsdir)

#Fiducial Cosmology
ps2_fid = {                                                                    
        "A_s" : 2.2321e-9,                                                      
        "n_s" : 0.967,                                                          
        "omega_b" : 0.02226,                                                    
        "omega_cdm" : 0.1127,                                                   
        "tau_reio" : 0.0598,                                                    
        "h" : 0.701,                                                            
        "T_cmb" : 2.726, # Units [K]                                            
        "N_ncdm" : 3.0,                                                          
        "deg_ncdm" : 1.0,                                                         
        "T_ncdm" : (1.95/2.726), # Units [T_cmb].                             
        "m_ncdm" : 0.02, # Units [eV]                                         
        "b0" : 1.0,                                                             
        "alphak2" : 1.0,                                                        
        "sigma_fog_0" : 250000, #Units [m s^-2]                                 
        "N_eff" : 3.046 - (3 * 1.0132), #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : None # Fix T_ncdm or m_ncdm                          
        }

# DESI survey parameters
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Run Fisher Forecast for cosmology
ps2_forecast = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    ps2_fid, 
    z_table, 
    "DESI",
    dNdz, 
    fcoverage_deg=skycover,
    dstep=derivative_step,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)
ps2_forecast.gen_pm()
ps2_forecast.gen_fisher(
    fisher_order=[
        'omega_b',                                    
        'omega_cdm',                                  
        'n_s',                                        
        'A_s',                                        
        'tau_reio',                                   
        'h',                                          
        'M_ncdm',                                                                  
        'sigma_fog',                                   
        'b0',                                         
        'alpha_k2'
    ],
    mu_step=mu_integral_step, 
    skipgen=False)
dill.dump_session(os.path.join(ps2_resultsdir, 'test2.db'))

In [ ]:
import dill
dill.load_session('/Volumes/SSD01/results/test2/test2.db')

ps2_resultsdir = cf.correct_path("/Volumes/SSD01/results/test2/")

#survey="CMBS4"
survey="Planck"

prepath = cf.priors_directory()
head = 'omega_b \t omega_cdm \t n_s \t A_s \t tau_reio \t h \t M_ncdm \t sigma_fog \t beta0 \t alpha_k2'

# Add CMB information
ps2_forecast.load_cmb_fisher(
        fisher_order=[
            'omega_b',                                    
            'omega_cdm',                                  
            'n_s',                                        
            'A_s',                                        
            'tau_reio',                                   
            'h',                                                                             
            'M_ncdm'],
        fisherpath=os.path.join(cf.priors_directory(), survey+"_Fisher_Neutrinos.dat"))

outdir=os.path.join(ps2_resultsdir, survey)
cf.makedirectory(outdir)
ps2_forecast.export_matrices(outdir)
np.savetxt(outdir + '/Full_Fisher.dat', ps2_forecast.numpy_full_fisher, delimiter='\t', header=head)
ps2_forecast.pandas_lss_fisher.to_csv(outdir + "/LSS_Fisher.dat", sep='\t', index=False)

# Test 3: 2relic Branch, gM Forecast

In [ ]:
import os                                                                       
import shutil                                                                   
import numpy as np                                                              
import pandas as pd                                                             
import seaborn as sns                                                           
import cosmicfish as cf                                                         
import matplotlib.pyplot as plt                                                 
import dill                                                                     
                                                                                
                                                                                
# Instruct pyplot to use seaborn                                                
sns.set()                                                                       
                                                                                
# Set project, data, CLASS directories                                                                                    
projectdir = '/Volumes/SSD01/results/test3/'                                    
datastore = '/Volumes/SSD01/data.nosynctest3a/'                                  
classpath = '/Users/nicholasdeporzio/Desktop/cfworkspace/class/'                
                                                                                
# Specify resolution of numerical integrals                                     
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 

# Generate output paths                                                                
ps23_resultsdir = projectdir                                                               
cf.makedirectory(ps23_resultsdir)                                                                                        
                                                                                
# Linda Fiducial Cosmology                                                      
ps23_fid = {                                                                    
        "A_s" : 2.2321e-9,                                                      
        "n_s" : 0.967,                                                          
        "omega_b" : 0.02226,                                                    
        "omega_cdm" : 0.1127,                                                   
        "tau_reio" : 0.0598,                                                    
        "h" : 0.701,                                                            
        "T_cmb" : 2.726, # Units [K]                                            
        "N_ncdm" : 4.,                                                          
        "deg_ncdm" : 1.0,                                                       
        "T_ncdm" : (0.79/2.726), # Units [T_cmb].                               
        "m_ncdm" : 0.01, # Units [eV]                                           
        "b0" : 1.0,                                                             
        "alphak2" : 1.0,                                                        
        "sigma_fog_0" : 250000, #Units [m s^-2]                                 
        "N_eff" : 0.0064, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm",  # Fix T_ncdm or m_ncdm                        
        "m_nu" : 0.02                                                           
        }    

# DESI Parameters                                                               
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2                         
                                                                                
# Run Fisher Forecast                                                                                               
masses = np.geomspace(0.01, 10., 21)                                                    
temps = np.array([0.79, 0.91, 0.94, 1.08])                                                        
                                                                                
omegacdm_set = np.array([                                                       
    ps23_fid['omega_cdm']                                                       
    - ((masses/cf.NEUTRINO_SCALE_FACTOR)* np.power(tval / 1.95, 3.))                                            
    for tidx, tval in enumerate(temps)])    

ps23_fiducialset = [[                                                           
    dict(ps23_fid, **{                                                          
        'm_ncdm' : masses[midx],                                                
        'omega_cdm' : omegacdm_set[tidx, midx],                                 
        'T_ncdm' : temps[tidx]/2.726})                                          
    for midx, mval in enumerate(masses)]                                        
    for tidx, tval in enumerate(temps)]   

ps23_forecastset = [[cf.forecast(                                               
    classpath,                                                                  
    datastore,                                                                  
    '2relic',                                                                   
    fidval,                                                                     
    z_table,                                                                    
    "DESI",                                                                     
    dNdz,                                                                       
    fcoverage_deg=skycover,                                                     
    dstep=derivative_step,                                                      
    RSD=True,                                                                   
    FOG=True,                                                                   
    AP=True,                                                                    
    COV=True)                                                                   
    for fididx, fidval in enumerate(fidrowvals)]                                
    for fidrowidx, fidrowvals in enumerate(ps23_fiducialset)] 

#dill.load_session('')                                                          
for frowidx, frowval in enumerate(ps23_forecastset):                            
    for fidx, fcst in enumerate(frowval):                                       
        if type(fcst.fisher)==type(None):                                       
            fcst.gen_pm()                                                       
            fcst.gen_fisher(                                                    
                fisher_order=[                                                  
                    'omega_b',                                                  
                    'omega_cdm',                                                
                    'n_s',                                                      
                    'A_s',                                                      
                    'tau_reio',                                                 
                    'h',                                                        
                    'N_ncdm',                                                   
                    'M_ncdm',                                                   
                    'sigma_fog',                                                
                    'b0',                                                       
                    'alpha_k2'],                                                
                mu_step=mu_integral_step,                                       
                skipgen=False)                                                  
            print("Relic Forecast ", fidx, " complete...")                      
            dill.dump_session(os.path.join(ps23_resultsdir, 'test3.db'))
        else:                                                                   
            print('Fisher matrix already generated!')  

In [ ]:
import dill
dill.load_session('/Volumes/SSD01/results/test3/test3_1.08K.db')
import cosmicfish as cf

ps23_resultsdir = cf.correct_path("/Volumes/SSD01/results/test3/")

#cmbsurvey="CMBS4"
cmbsurvey="Planck"

prepath = cf.priors_directory()
tempnames=['1.08']
#head = 'A_s \t h \t n_s \t omega_b \t omega_cdm \t tau_reio \t M_ncdm \t N_ncdm \t sigma_fog \t beta0 \t alpha_k2'


for tidx, tval in enumerate(ps23_forecastset):
    for midx, mval in enumerate(tval): 
        mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'M_ncdm', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
        mval.load_cmb_fisher(
              fisher_order=[
                'A_s',
                'h',  
                'n_s',
                'omega_b',                                    
                'omega_cdm',                                                     
                'tau_reio',                                                                                    
                'M_ncdm', 
                'N_ncdm'],
            fisherpath = cf.priors_directory()+'/2relic_Priors/Fisher_T'+tempnames[tidx]+'K/Fisher'+cmbsurvey+'_bin'+str(midx+1)+'_T'+tempnames[tidx]+'K.txt'
        )
        outdir=os.path.join(ps23_resultsdir, cmbsurvey, 'T'+tempnames[tidx], 'M'+str(midx+1))
        cf.makedirectory(outdir)
        mval.export_matrices(outdir)
        #np.savetxt(outdir + '_Full_Fisher.dat', mval.numpy_full_fisher, delimiter='\t', header=head)
        #mval.pandas_lss_fisher.to_csv(outdir + "_LSS_Fisher.dat", sep='\t', index=False)

# Test 4: Testing High-Precision CLASS Output

## Generate High Precision Data

In [ ]:
import os                                                                       
import shutil                                                                   
import numpy as np                                                              
import pandas as pd                                                             
import seaborn as sns                                                           
import cosmicfish as cf                                                         
import matplotlib.pyplot as plt                                                 
import dill                                                                     
                                                                                
                                                                                
# Instruct pyplot to use seaborn                                                
sns.set()                                                                       
                                                                                
# Set project, data, CLASS directories                                                                                    
projectdir = '/Volumes/SSD01/results/test4/'                                    
datastore = '/Volumes/SSD01/data.nosynctest4/'                                  
classpath = '/Users/nicholasdeporzio/Desktop/cfworkspace/class/'                
                                                                                
# Specify resolution of numerical integrals                                     
derivative_step = 0.008 # How much to vary parameter to calculate numerical derivative
#mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 
mu_integral_step = 1.00

# Generate output paths                                                                
ps23_resultsdir = projectdir                                                               
cf.makedirectory(ps23_resultsdir)                                                                                        
                                                                                
# Linda Fiducial Cosmology                                                      
ps23_fid = {                                                                    
        "A_s" : 2.2321e-9,                                                      
        "n_s" : 0.967,                                                          
        "omega_b" : 0.02226,                                                    
        "omega_cdm" : 0.1127,                                                   
        "tau_reio" : 0.0598,                                                    
        "h" : 0.701,                                                            
        "T_cmb" : 2.726, # Units [K]                                            
        "N_ncdm" : 4.,                                                          
        "deg_ncdm" : 1.0,                                                       
        "T_ncdm" : (0.79/2.726), # Units [T_cmb].                               
        "m_ncdm" : 0.01, # Units [eV]                                           
        "b0" : 1.0,                                                             
        "alphak2" : 1.0,                                                        
        "sigma_fog_0" : 250000, #Units [m s^-2]                                 
        "N_eff" : 0.0064, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm",  # Fix T_ncdm or m_ncdm                        
        "m_nu" : 0.02                                                           
        }    

# DESI Parameters                                                               
z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2                         
                                                                                
# Run Fisher Forecast                                                                                               
#masses = np.geomspace(0.01, 10., 21)                                                    
#temps = np.array([0.79, 0.91, 0.94, 1.08])     
masses = np.array([np.geomspace(0.01, 10., 21)[0], np.geomspace(0.01, 10., 21)[13], np.geomspace(0.01, 10., 21)[-1]])
temps = np.array([0.79, 1.08])
                                                                                
omegacdm_set = np.array([                                                       
    ps23_fid['omega_cdm']                                                       
    - ((masses/cf.NEUTRINO_SCALE_FACTOR)* np.power(tval / 1.95, 3.))                                            
    for tidx, tval in enumerate(temps)])    

ps23_fiducialset = [[                                                           
    dict(ps23_fid, **{                                                          
        'm_ncdm' : masses[midx],                                                
        'omega_cdm' : omegacdm_set[tidx, midx],                                 
        'T_ncdm' : temps[tidx]/2.726})                                          
    for midx, mval in enumerate(masses)]                                        
    for tidx, tval in enumerate(temps)]   

ps23_forecastset = [[cf.forecast(                                               
    classpath,                                                                  
    datastore,                                                                  
    '2relic',                                                                   
    fidval,                                                                     
    z_table,                                                                    
    "DESI",                                                                     
    dNdz,                                                                       
    fcoverage_deg=skycover,                                                     
    dstep=derivative_step,                                                      
    RSD=False,                                                                   
    FOG=False,                                                                   
    AP=False,                                                                    
    COV=False)                                                                   
    for fididx, fidval in enumerate(fidrowvals)]                                
    for fidrowidx, fidrowvals in enumerate(ps23_fiducialset)] 

#dill.load_session('')                                                          
for frowidx, frowval in enumerate(ps23_forecastset):                            
    for fidx, fcst in enumerate(frowval):                                       
        if type(fcst.fisher)==type(None):                                       
            fcst.gen_pm()                                                       
            fcst.gen_fisher(                                                    
                fisher_order=[                                                  
                    'omega_b',                                                  
                    'omega_cdm',                                                
                    'n_s',                                                      
                    'A_s',                                                      
                    'tau_reio',                                                 
                    'h',                                                        
                    'N_ncdm',                                                   
                    'M_ncdm',                                                   
                    'sigma_fog',                                                
                    'b0',                                                       
                    'alpha_k2'],                                                
                mu_step=mu_integral_step,                                       
                skipgen=False)                                                  
            print("Relic Forecast ", fidx, " complete...")                      
            dill.dump_session(os.path.join(ps23_resultsdir, 'test4_pk_ref.db'))
        else:                                                                   
            print('Fisher matrix already generated!')  

## Compare default precision profile with HP cl_permille.pre profile 

In [ ]:
import dill 
dill.load_session('/Volumes/SSD01/results/test4/test4_permille.db')
sns.set()
sns.set_palette('bright')

#mass = '0.010'
mass = '0.891'
temp = '0.79'
param = 'N_ncdm'
data = 'cl'
midx = 13
tidx = 0 
red_midx=1
red_tidx=0

default_path_hi = '/Volumes/SSD01/Cls/T'+temp+'K/M'+mass+'eV/'+param+'/high/test_'+data+'.dat'
default_path_lo = '/Volumes/SSD01/Cls/T'+temp+'K/M'+mass+'eV/'+param+'/low/test_'+data+'.dat'
hp_path_hi = (eval(('ps23_forecastset['+str(red_tidx)+']['+str(red_midx)+'].'+param+'_high[0].datapath'))[0:-6]+data+'.dat')
hp_path_lo = (eval(('ps23_forecastset['+str(red_tidx)+']['+str(red_midx)+'].'+param+'_low[0].datapath'))[0:-6]+data+'.dat')

print(default_path_hi)
print(hp_path_hi)

default_l_hi = np.loadtxt(default_path_hi, skiprows=12, usecols={0})
default_TT_hi = np.loadtxt(default_path_hi, skiprows=12, usecols={1})
default_EE_hi = np.loadtxt(default_path_hi, skiprows=12, usecols={2})
default_TE_hi = np.loadtxt(default_path_hi, skiprows=12, usecols={3})
default_l_lo = np.loadtxt(default_path_lo, skiprows=12, usecols={0})
default_TT_lo = np.loadtxt(default_path_lo, skiprows=12, usecols={1})
default_EE_lo = np.loadtxt(default_path_lo, skiprows=12, usecols={2})
default_TE_lo = np.loadtxt(default_path_lo, skiprows=12, usecols={3})

hp_l_hi = np.loadtxt(hp_path_hi, skiprows=12, usecols={0})
hp_TT_hi = np.loadtxt(hp_path_hi, skiprows=12, usecols={1})
hp_EE_hi = np.loadtxt(hp_path_hi, skiprows=12, usecols={2})
hp_TE_hi = np.loadtxt(hp_path_hi, skiprows=12, usecols={3})
hp_l_lo = np.loadtxt(hp_path_lo, skiprows=12, usecols={0})
hp_TT_lo = np.loadtxt(hp_path_lo, skiprows=12, usecols={1})
hp_EE_lo = np.loadtxt(hp_path_lo, skiprows=12, usecols={2})
hp_TE_lo = np.loadtxt(hp_path_lo, skiprows=12, usecols={3})

scaleEE = 100
scaleTE = 100

plt.figure(figsize=(15,7.5))
plt.semilogx(default_l_hi, default_TT_hi, linestyle='solid', linewidth=3, label='Default, TT')
plt.semilogx(hp_l_hi, hp_TT_hi, linestyle='dashed', label='HP, TT')
plt.semilogx(default_l_hi, scaleEE*default_EE_hi, linestyle='solid', linewidth=3, label='Default, EE')
plt.semilogx(hp_l_hi, scaleEE*hp_EE_hi, linestyle='dashed', label='HP, EE')
plt.semilogx(default_l_hi, scaleTE*default_TE_hi, linestyle='solid', linewidth=3, label='Default, TE')
plt.semilogx(hp_l_hi, scaleTE*hp_TE_hi, linestyle='dashed', label='HP, TE')
plt.xlabel(r"l", fontsize=24)                                       
plt.ylabel(r"TT", fontsize=24)
plt.title(r'Power Spectrum (High)', fontsize=24)
plt.legend(fontsize=18, loc='upper right')                                       
plt.tick_params(axis='x', which='minttor')   
plt.xlim(100, 5000)
plt.grid(True, which='minor')                                                   
#plt.savefig(prepath+'Plots/T'+str(temp)+'K/'+'Errors_CMBS4_Only_'+str(temp)+'K.png')
plt.show()

plt.figure(figsize=(15,7.5))
plt.semilogx(default_l_hi, default_TT_hi-default_TT_lo, linestyle='solid', linewidth=3, label='Default, TT')
plt.semilogx(hp_l_hi, hp_TT_hi-hp_TT_lo, linestyle='dashed', label='HP, TT')
plt.semilogx(default_l_hi, scaleEE*(default_EE_hi-default_EE_lo), linestyle='solid', linewidth=3, label='Default, EE')
plt.semilogx(hp_l_hi, scaleEE*(hp_EE_hi-hp_EE_lo), linestyle='dashed', label='HP, EE')
plt.semilogx(default_l_hi, scaleTE*(default_TE_hi-default_TE_lo), linestyle='solid', linewidth=3, label='Default, TE')
plt.semilogx(hp_l_hi, scaleTE*(hp_TE_hi-hp_TE_lo), linestyle='dashed', label='HP, TE')
plt.xlabel(r"l", fontsize=24)                                       
plt.ylabel(r"TT", fontsize=24)
plt.title(r'Delta Power Spectum ', fontsize=24)
plt.legend(fontsize=18, loc='upper right')                                       
plt.tick_params(axis='x', which='minttor')   
plt.xlim(100, 5000)
plt.grid(True, which='minor')                                                   
#plt.savefig(prepath+'Plots/T'+str(temp)+'K/'+'Errors_CMBS4_Only_'+str(temp)+'K.png')
plt.show()

# Test 5: Testing Convergence 

## Generate Convergence Forecasts

In [ ]:
import os                                                                       
import shutil                                                                   
import numpy as np                                                              
import pandas as pd                                                             
import seaborn as sns                                                           
import cosmicfish as cf                                                         
import matplotlib.pyplot as plt                                                 
import dill                                                                     
                                                                                
                                                                                
# Instruct pyplot to use seaborn                                                
sns.set()                                                                       
                                                                                
# Set project, data, CLASS directories                                                                                    
projectdir = '/Volumes/SSD01/results/test5/'                                    
datastore = '/Volumes/SSD01/data.nosynctest5/'                                  
classpath = '/Users/nicholasdeporzio/Desktop/cfworkspace/class/'                
                                                                                
# Specify resolution of numerical integrals                                     
derivative_steps = [0.05, 0.1, 0.15] # How much to vary parameter to calculate numerical derivative
#mu_integral_step = 0.05 # For calculating numerical integral wrt mu between -1 and 1 
mu_integral_step = 1.00

# Generate output paths                                                                
ps23_resultsdir = projectdir                                                               
cf.makedirectory(ps23_resultsdir)                                                                                        
                                                                                
# Linda Fiducial Cosmology                                                      
ps23_fid = {                                                                    
        "A_s" : 2.2321e-9,                                                      
        "n_s" : 0.967,                                                          
        "omega_b" : 0.02226,                                                    
        "omega_cdm" : 0.1127,                                                   
        "tau_reio" : 0.0598,                                                    
        "h" : 0.701,                                                            
        "T_cmb" : 2.726, # Units [K]                                            
        "N_ncdm" : 4.,                                                          
        "deg_ncdm" : 1.0,                                                       
        "T_ncdm" : (0.79/2.726), # Units [T_cmb].                               
        "m_ncdm" : 0.01, # Units [eV]                                           
        "b0" : 1.0,                                                             
        "alphak2" : 1.0,                                                        
        "sigma_fog_0" : 250000, #Units [m s^-2]                                 
        "N_eff" : 0.0064, #We allow relativistic neutrinos in addition to our DM relic
        "relic_vary" : "N_ncdm",  # Fix T_ncdm or m_ncdm                        
        "m_nu" : 0.02                                                           
        }    

# DESI Parameters                                                               
#z_table = np.array([0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85])
#dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
z_table = np.array([0.65, 0.75])
dNdz = np.array([309., 2269.])
skycover = 14000. # Sky coverage of survey in degrees^2                         
                                                                                
# Run Fisher Forecast                                                                                               
#masses = np.geomspace(0.01, 10., 21)                                                    
#temps = np.array([0.79, 0.91, 0.94, 1.08])    
masses = np.array([
    np.geomspace(0.01, 10., 21)[0],
    np.geomspace(0.01, 10., 21)[1],
    np.geomspace(0.01, 10., 21)[13],
    np.geomspace(0.01, 10., 21)[-2],
    np.geomspace(0.01, 10., 21)[-1]])
temps = np.array([0.79, 1.08])
                                                                                
omegacdm_set = np.array([                                                       
    ps23_fid['omega_cdm']                                                       
    - ((masses/cf.NEUTRINO_SCALE_FACTOR)* np.power(tval / 1.95, 3.))                                            
    for tidx, tval in enumerate(temps)])    

ps23_fiducialset = [[[                                                           
    dict(ps23_fid, **{                                                          
        'm_ncdm' : masses[midx],                                                
        'omega_cdm' : omegacdm_set[tidx, midx],                                 
        'T_ncdm' : temps[tidx]/2.726})                                          
    for midx, mval in enumerate(masses)]                                        
    for tidx, tval in enumerate(temps)]  
    for didx, dval in enumerate(derivative_steps)]

ps23_forecastset = [[[cf.forecast(                                               
    classpath,                                                                  
    datastore,                                                                  
    '2relic',                                                                   
    fidval,                                                                     
    z_table,                                                                    
    "DESI",                                                                     
    dNdz,                                                                       
    fcoverage_deg=skycover,                                                     
    dstep=0.008,
    gstep=derivative_steps[didx],
    RSD=False,                                                                   
    FOG=False,                                                                   
    AP=False,                                                                    
    COV=False)                                                                   
    for fididx, fidval in enumerate(fidrowvals)]                                
    for fidrowidx, fidrowvals in enumerate(dval)] 
    for didx, dval in enumerate(ps23_fiducialset)]

#dill.load_session('')     
for didx, dval in enumerate(ps23_forecastset): 
    for frowidx, frowval in enumerate(dval):                            
        for fidx, fcst in enumerate(frowval):                                       
            if type(fcst.fisher)==type(None):                                       
                fcst.gen_pm()                                                       
                fcst.gen_fisher(                                                    
                    fisher_order=[                                                  
                        'omega_b',                                                  
                        'omega_cdm',                                                
                        'n_s',                                                      
                        'A_s',                                                      
                        'tau_reio',                                                 
                        'h',                                                        
                        'N_ncdm',                                                   
                        'M_ncdm',                                                   
                        'sigma_fog',                                                
                        'b0',                                                       
                        'alpha_k2'],                                                
                    mu_step=mu_integral_step,                                       
                    skipgen=False)                                                  
                print("Relic Forecast ", fidx, " complete...")                      
                dill.dump_session(os.path.join(ps23_resultsdir, 'test5'+'_'+str(didx)+'_'+str(frowidx)+'_'+str(fidx)+'.db'))
            else:                                                                   
                print('Fisher matrix already generated!')  

## Make Convergence Plots

In [ ]:
import dill 
dill.load_session('/Volumes/SSD01/results/test5/test5_2_1_4.db')
sns.set()
sns.set_palette('bright')

dsteps = [0.05, 0.1, 0.15]
masses = np.array([
    np.geomspace(0.01, 10., 21)[0],
    np.geomspace(0.01, 10., 21)[1],
    np.geomspace(0.01, 10., 21)[13],
    np.geomspace(0.01, 10., 21)[-2],
    np.geomspace(0.01, 10., 21)[-1]])
temps = np.array([0.79, 1.08])

var = 'N_ncdm'
varfid = 1.0

midx=4
tidx=0

k = [eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_high[0].k_table') for didx, dval in enumerate(dsteps)]
pm_hi = [eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_high[0].ps_table') for didx, dval in enumerate(dsteps)]
pb_hi = [eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_high[0].b_interp_table') for didx, dval in enumerate(dsteps)]
pc_hi = [eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_high[0].cdm_interp_table') for didx, dval in enumerate(dsteps)]
pm_lo = [eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_low[0].ps_table') for didx, dval in enumerate(dsteps)]
pb_lo = [eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_low[0].b_interp_table') for didx, dval in enumerate(dsteps)]
pc_lo = [eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_low[0].cdm_interp_table') for didx, dval in enumerate(dsteps)]
dpm = [(pm_hi[didx]-pm_lo[didx])/(2.*dval*varfid) for didx, dval in enumerate(dsteps)]
dpb = [(pb_hi[didx]-pb_lo[didx])/(2.*dval*varfid) for didx, dval in enumerate(dsteps)]
dpc = [(pc_hi[didx]-pc_lo[didx])/(2.*dval*varfid) for didx, dval in enumerate(dsteps)]

cl_l = [np.loadtxt(eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_high[0].datadirectory')+'/test_cl.dat', skiprows=12, usecols={0}) for didx, dval in enumerate(dsteps)]
TT_hi = [np.loadtxt(eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_high[0].datadirectory')+'/test_cl.dat', skiprows=12, usecols={1}) for didx, dval in enumerate(dsteps)]
TT_lo = [np.loadtxt(eval('ps23_forecastset['+str(didx)+']['+str(tidx)+']['+str(midx)+'].'+var+'_low[0].datadirectory')+'/test_cl.dat', skiprows=12, usecols={1}) for didx, dval in enumerate(dsteps)]
dTT = [(TT_hi[didx]-TT_lo[didx])/(2.*dval*varfid) for didx, dval in enumerate(dsteps)]


####
# Pm 
####
plt.figure(figsize=(15,7.5))
for didx, dval in enumerate(dsteps): 
    plt.semilogx(k[didx], dpm[didx], linestyle='solid', linewidth=3, label=r'$\Delta\theta=$'+str(100.*dval)+r'%')
plt.xlabel(r"k", fontsize=24)                                       
plt.ylabel(r"$dP_m/d\theta$", fontsize=24)
plt.title(r'T='+str(temps[tidx])+', M='+str(masses[midx]),  fontsize=24)
plt.legend(fontsize=18, loc='upper right')                                       
plt.tick_params(axis='x', which='minttor')   
#plt.xlim(100, 5000)
plt.grid(True, which='minor')                                                   
plt.savefig('/Users/nicholasdeporzio/Desktop/test/p1.png')
plt.show()

####
# Pb 
####
#plt.figure(figsize=(15,7.5))
#for didx, dval in enumerate(dsteps): 
#    plt.semilogx(k[didx], dpb[didx], linestyle='solid', linewidth=3, label=r'$\Delta\theta=$'+str(100.*dval)+r'%')
#plt.xlabel(r"k", fontsize=24)                                       
#plt.ylabel(r"$dP_b/d\theta$", fontsize=24)
#plt.title(r'T='+str(temps[tidx])+', M='+str(masses[midx]),  fontsize=24)
#plt.legend(fontsize=18, loc='upper right')                                       
#plt.tick_params(axis='x', which='minttor')   
##plt.xlim(100, 5000)
#plt.grid(True, which='minor')                                                   
##plt.savefig(prepath+'Plots/T'+str(temp)+'K/'+'Errors_CMBS4_Only_'+str(temp)+'K.png')
#plt.show()

####
# Pc 
####
#plt.figure(figsize=(15,7.5))
#for didx, dval in enumerate(dsteps): 
#    plt.semilogx(k[didx], dpc[didx], linestyle='solid', linewidth=3, label=r'$\Delta\theta=$'+str(100.*dval)+r'%')
#plt.xlabel(r"k", fontsize=24)                                       
#plt.ylabel(r"$dP_c/d\theta$", fontsize=24)
#plt.title(r'T='+str(temps[tidx])+', M='+str(masses[midx]),  fontsize=24)
#plt.legend(fontsize=18, loc='upper right')                                       
#plt.tick_params(axis='x', which='minttor')   
##plt.xlim(100, 5000)
#plt.grid(True, which='minor')                                                   
##plt.savefig(prepath+'Plots/T'+str(temp)+'K/'+'Errors_CMBS4_Only_'+str(temp)+'K.png')
#plt.show()

####
# TT
####
plt.figure(figsize=(15,7.5))
for didx, dval in enumerate(dsteps): 
    plt.semilogx(cl_l[didx], dTT[didx], linestyle='solid', linewidth=3, label=r'$\Delta\theta=$'+str(100.*dval)+r'%')
plt.xlabel(r"l", fontsize=24)                                       
plt.ylabel(r"$dTT/d\theta$", fontsize=24)
plt.title(r'T='+str(temps[tidx])+', M='+str(masses[midx]),  fontsize=24)
plt.legend(fontsize=18, loc='upper right')                                       
plt.tick_params(axis='x', which='minttor')   
#plt.xlim(100, 5000)
plt.grid(True, which='minor')                                                   
plt.savefig('/Users/nicholasdeporzio/Desktop/test/p2.png')
plt.show()

####
# deltaTT
####
plt.figure(figsize=(15,7.5))
for didx, dval in enumerate(dsteps): 
    plt.semilogx(cl_l[didx], (dTT[didx]-dTT[-2])/dTT[-2], linestyle='solid', linewidth=3, label=r'$\Delta\theta=$'+str(100.*dval)+r'%')
plt.xlabel(r"l", fontsize=24)                                       
plt.ylabel(r"$\Delta(dTT/d\theta)/(dTT/d\theta_{10\%})$", fontsize=24)
plt.title(r'T='+str(temps[tidx])+', M='+str(masses[midx]),  fontsize=24)
plt.legend(fontsize=18, loc='upper right')                                       
plt.tick_params(axis='x', which='minttor')   
plt.ylim(-5, 5)
plt.grid(True, which='minor')                                                   
plt.savefig('/Users/nicholasdeporzio/Desktop/test/p3.png')
plt.show()

####
# Pm 
####
plt.figure(figsize=(15,7.5))
for didx, dval in enumerate(dsteps): 
    plt.semilogx(k[didx], (dpm[didx]-dpm[-2])/dpm[-2], linestyle='solid', linewidth=3, label=r'$\Delta\theta=$'+str(100.*dval)+r'%')
plt.xlabel(r"k", fontsize=24)                                       
plt.ylabel(r"$\Delta(dP_m/d\theta)/(dP_m/d\theta_{10\%})$", fontsize=24)
plt.title(r'T='+str(temps[tidx])+', M='+str(masses[midx]),  fontsize=24)
plt.legend(fontsize=18, loc='upper right')                                       
plt.tick_params(axis='x', which='minttor')   
#plt.xlim(100, 5000)
plt.grid(True, which='minor')                                                   
plt.savefig('/Users/nicholasdeporzio/Desktop/test/p4.png')
plt.show()





# Plot Contours

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

#dataset = 'Full'
dataset = 'LSS'

#survey = 'CMBS4'
survey = 'Planck'

#Import fishers
fisher_1 = np.loadtxt('/Volumes/SSD01/results/test1/'+survey+'/T0.91/M1_'+dataset+'_Fisher.dat', skiprows=1)
fisher_2 = np.loadtxt('/Volumes/SSD01/results/test2/'+survey+'/'+dataset+'_Fisher.dat', skiprows=1)
fisher_3 = np.loadtxt('/Volumes/SSD01/results/test3/'+survey+'/T0.91/M1_'+dataset+'_Fisher.dat', skiprows=1)

fisher_1 = np.loadtxt('/Volumes/SSD01/results/test1/'+survey+'/T0.91/M1_'+dataset+'_Fisher.dat', skiprows=1)
fisher_2 = np.loadtxt('/Volumes/SSD01/results/test2/'+survey+'/'+dataset+'_Fisher.dat', skiprows=1)
fisher_3 = np.loadtxt('/Volumes/SSD01/results/test3/'+survey+'/T0.91/M1_'+dataset+'_Fisher.dat', skiprows=1)

#Delete tau and invert for covariances
cov_1 = np.linalg.inv(np.delete(np.delete(fisher_1, 5, axis=0), 5, axis=1)) 
cov_2 = np.linalg.inv(np.delete(np.delete(fisher_2, 4, axis=0), 4, axis=1)) 
cov_3 = np.linalg.inv(np.delete(np.delete(fisher_3, 5, axis=0), 5, axis=1)) 

fid_1 = [
    2.2321e-9, #A_s
    0.701, #h
    0.967, #n_s
    0.02226, #omega_b
    0.1127, #omega_cdm
#    0.0598, #tau_reio
    1.0, #N_ncdm 
    250., #sigma_fog
    1.0, #beta0
    1.0, #alphak2 
]
fid_2 = [
    0.02226, #omega_b
    0.1127, #omega_cdm
    0.967, #n_s
    2.2321e-9, #A_s
#    0.0598, #tau_reio
    0.701, #h
    0.06, #M_ncdm 
    250., #sigma_fog
    1.0, #beta0
    1.0, #alphak2 
]
fid_3 = [
    2.2321e-9, #A_s
    0.701, #h
    0.967, #n_s
    0.02226, #omega_b
    0.1127, #omega_cdm
#    0.0598, #tau_reio
    1.0, #N_ncdm
    0.06, #M_ncdm
    250., #sigma_fog
    1.0, #beta0
    1.0, #alphak2 
]



params_1 = [
    r'$A_s$', 
    r'$h$', 
    r'$n_s$', 
    r'$\omega_b$', 
    r'$\omega_{cdm}$', 
#    r'$\tau_{reio}$', 
    r'$g_\chi$', 
    r'$\sigma_{fog}$', 
    r'$\beta_0$', 
    r'$\alpha_{k2}$']
params_2 = [
    r'$\omega_b$', 
    r'$\omega_{cdm}$', 
    r'$n_s$', 
    r'$A_s$',
#    r'$\tau_{reio}$',
    r'$h$',  
    r'$M_\nu$', 
    r'$\sigma_{fog}$', 
    r'$\beta_0$', 
    r'$\alpha_{k2}$']
params_3 = [
    r'$A_s$', 
    r'$h$', 
    r'$n_s$', 
    r'$\omega_b$', 
    r'$\omega_{cdm}$', 
#    r'$\tau_{reio}$', 
    r'$g_\chi$', 
    r'$M_\nu$', 
    r'$\sigma_{fog}$', 
    r'$\beta_0$', 
    r'$\alpha_{k2}$']

gauss_1 = GaussianND(fid_1, cov_1, names=params_1)
gauss_2 = GaussianND(fid_2, cov_2, names=params_2)
gauss_3 = GaussianND(fid_3, cov_3, names=params_3)

h = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
h.settings.figure_legend_frame = False #Box around legend
h.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
h.settings.axes_fontsize=14
h.settings.axes_labelsize=20
h.settings.axis_tick_x_rotation=45
h.settings.legend_fontsize=20
#h.settings.axis_tick_y_rotation=90
#h.settings.colorbar_tick_rotation=90
if dataset=='Full': 
    if survey=='CMBS4':
        legendposttext = ', DESI+CMBS4'
    elif survey=='Planck':
        legendposttext = ', DESI+Planck'
elif dataset=='LSS': 
    legendposttext = ', DESI Only'
else: 
    legendposttext = ''
h.triangle_plot(
    [
        gauss_1, 
        #gauss_2, 
        gauss_3
    ], 
    filled=True,
    legend_labels=[
        r'$g_\chi$-Only'+legendposttext, 
        #r'$M_\nu$ -Only'+legendposttext, 
        r'$g_\chi$ & $M_\nu$ Simultaneous' + legendposttext
    ], 
    line_args=[
        {'ls':'solid', 'lw':2, 'color':'#A51C30'},
        #{'ls':'solid', 'lw':2, 'color':'#93a1ad'}, 
        {'ls':'solid', 'lw':2, 'color':'blue'}
    ], 
    contour_args=[
        {'ls':'dotted', 'lw':2, 'color':'#A51C30', 'filled' : True},
        #{'ls':'dotted', 'lw':2, 'color':'#93a1ad', 'filled' : True}, 
        {'ls':'dotted', 'lw':2, 'color':'blue', 'filled' : False}
    ])
plt.savefig("/Users/nicholasdeporzio/Desktop/diagnostics/comparison.png")

# Add Priors to Cluster Output

In [ ]:
import os
import shutil
import dill
import numpy as np

#dtype=['Fine', 'Grid']
dtype=['Fine']
#lsssurvey=['BOSS', 'EUCLID', 'DESI']
lsssurvey=['DESI']
cmbsurvey=['CMBS4', 'Planck']

for idx1, val1 in enumerate(dtype):
    for idx2, val2 in enumerate(lsssurvey):
        for idx3, val3 in enumerate(cmbsurvey):
            prepath_in='/Volumes/SSD01/TEST_RESULTS_1/'+val2+'/Forecast_Files/'+val2+'_'+val1+'Plot_'
            prepath_out='/Volumes/SSD01/TEST_RESULTS_1/'+val2+'/Fisher_Matrices/'+val1+'/T'
            
            if val1=='Grid': 
                nmax=120
                modnum=10
                fishprefix='gp'
            
            elif val1=='Fine':
                nmax=84
                modnum=21
                fishprefix='fp'
                masses = np.geomspace(0.01, 10., 21)
                temps = np.array([0.79, 0.91, 0.94, 1.08])
            
            
            for idx in range(nmax): 
            
                temp_index=(idx // modnum)
                mass_index=(idx % modnum)
            
                dill.load_session(prepath_in+str(idx)+'/'+fishprefix+'_'+str(temp_index)+'_'+str(mass_index)+'.db')  
                
                if val1=='Grid':
                    masses = np.geomspace(0.01, 10., 10)
                    temps = np.linspace(0.79, 1.5, 12)
                elif val1=='Fine':
                    masses = np.geomspace(0.01, 10., 21)
                    temps = np.array([0.79, 0.91, 0.94, 1.08])
                
                #print(eval(fishprefix+'_forecastset')[0][0].fisher)
            
                mass = masses[mass_index]
                temp = temps[temp_index]
                
                outdir = prepath_out+f'{temp:.2f}'+'K/M'+f'{mass:.3f}'+'/'+val3+'/'
                cf.makedirectory(outdir)
            
                for tidx, tval in enumerate(eval(fishprefix+'_forecastset')):
                    for midx, mval in enumerate(tval): 
                        if val2=='DESI': 
                            mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'M_ncdm', 'N_ncdm', 'sigma_fog', 'b0', 'alpha_k2'])
                        else: 
                            mval.reorder_fisher(['A_s', 'h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'M_ncdm', 'N_ncdm', 'sigma_fog', 'beta0', 'beta1', 'alpha_k2'])
                            
                        mval.load_cmb_fisher(
                            fisher_order=[
                                'A_s',
                                'h',  
                                'n_s',
                                'omega_b',                                    
                                'omega_cdm',                                                     
                                'tau_reio',                                                                                    
                                'M_ncdm', 
                                'N_ncdm'],
                            fisherpath = cf.priors_directory()+'/2relic_Priors/'+val1+'/Fisher_T'+f'{temp:.2f}'+'K/Fisher'+val3+'_bin'+str(mass_index+1)+'_T'+f'{temp:.2f}'+'K.txt'
                        )
                        mval.export_matrices(outdir)



In [ ]:
import pandas as pd
import numpy as np
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=['a', 'b', 'c'])

test = np.array([df2.values, 10*df2.values])
np.shape(test)

# Plot Errors

## Forecast Class Definition

In [ ]:
import pandas as pd
import numpy as np 
from os.path import join as jn

class parameter:    
    def __init__(self, name, temp_vals=0, mass_vals=0):
        self.name = name
        self.fixed = False 

class forecast: 
    '''Forecasts Fisher/Covariance matrices from fid cosmo.'''
    
    def __init__(self, 
                 lss_survey_name, 
                 fisher_matrices_dir, 
                 masses, 
                 temps):
        
        self.lss_survey_name = lss_survey_name
        self.fisher_matrices_dir = fisher_matrices_dir
        self.masses = masses
        self.temps = temps
        
        self.prepath = jn(fisher_matrices_dir, 'Fine', 'T')
        
        self.paths_lss = [[jn(self.prepath+str(tval)+'K', 'M'+f'{mval:.3f}', 'Planck', 'inv_lssfisher.mat') 
            for midx, mval in enumerate(masses)] 
            for tidx, tval in enumerate(temps)]
        self.paths_Planck = [[jn(self.prepath+str(tval)+'K', 'M'+f'{mval:.3f}', 'Planck', 'inv_cmbfisher.mat') 
            for midx, mval in enumerate(masses)] 
            for tidx, tval in enumerate(temps)]
        self.paths_CMBS4 = [[jn(self.prepath+str(tval)+'K', 'M'+f'{mval:.3f}', 'CMBS4', 'inv_cmbfisher.mat') 
            for midx, mval in enumerate(masses)] 
            for tidx, tval in enumerate(temps)]
        self.paths_Full_Planck = [[jn(self.prepath+str(tval)+'K', 'M'+f'{mval:.3f}', 'Planck', 'inv_fullfisher.mat') 
            for midx, mval in enumerate(masses)] 
            for tidx, tval in enumerate(temps)]
        self.paths_Full_CMBS4 = [[jn(self.prepath+str(tval)+'K', 'M'+f'{mval:.3f}', 'CMBS4', 'inv_fullfisher.mat') 
            for midx, mval in enumerate(masses)] 
            for tidx, tval in enumerate(temps)]
        
        self.covs_lss = [[pd.read_csv(self.paths_lss[tidx][midx], sep='\t', header=0)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.covs_Planck = [[pd.read_csv(self.paths_Planck[tidx][midx], sep='\t', header=0)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.covs_CMBS4 = [[pd.read_csv(self.paths_CMBS4[tidx][midx], sep='\t', header=0)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.covs_Full_Planck = [[pd.read_csv(self.paths_Full_Planck[tidx][midx], sep='\t', header=0)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.covs_Full_CMBS4 = [[pd.read_csv(self.paths_Full_CMBS4[tidx][midx], sep='\t', header=0)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        
        self.column_names = self.covs_Full_Planck[0][0].columns
        self.parameters = [parameter(mval) for midx, mval in enumerate(list(self.column_names))]

        self.fishers_lss = [[pd.DataFrame(np.linalg.inv(self.covs_lss[tidx][midx].values), columns=self.covs_lss[tidx][midx].columns)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.fishers_Planck = [[pd.DataFrame(np.linalg.inv(self.covs_Planck[tidx][midx].values), columns=self.covs_Planck[tidx][midx].columns)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.fishers_CMBS4 = [[pd.DataFrame(np.linalg.inv(self.covs_CMBS4[tidx][midx].values), columns=self.covs_CMBS4[tidx][midx].columns)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.fishers_Full_Planck = [[pd.DataFrame(np.linalg.inv(self.covs_Full_Planck[tidx][midx].values), columns=self.covs_Full_Planck[tidx][midx].columns)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.fishers_Full_CMBS4 = [[pd.DataFrame(np.linalg.inv(self.covs_Full_CMBS4[tidx][midx].values), columns=self.covs_Full_CMBS4[tidx][midx].columns)
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]

        self.clear()
                
    def clear(self): 
        for pidx, pval in enumerate(self.parameters): 
            pval.fixed=False 
            
        self.final_covs_lss = [[pd.DataFrame(self.covs_lss[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_covs_Planck = [[pd.DataFrame(self.covs_Planck[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_covs_CMBS4 = [[pd.DataFrame(self.covs_CMBS4[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_covs_Full_Planck = [[pd.DataFrame(self.covs_Full_Planck[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_covs_Full_CMBS4 = [[pd.DataFrame(self.covs_Full_CMBS4[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]

        self.final_fishers_lss = [[pd.DataFrame(self.fishers_lss[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_fishers_Planck = [[pd.DataFrame(self.fishers_Planck[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_fishers_CMBS4 = [[pd.DataFrame(self.fishers_CMBS4[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_fishers_Full_Planck = [[pd.DataFrame(self.fishers_Full_Planck[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        self.final_fishers_Full_CMBS4 = [[pd.DataFrame(self.fishers_Full_CMBS4[tidx][midx])
            for midx, mval in enumerate(masses)]
            for tidx, tval in enumerate(temps)]
        
    def fix_param(self, name): 
        flag=False
        for pidx, pval in enumerate(self.parameters):
            if pval.name==name:
                pval.fixed=True
                flag=True
        if flag==True: 
            self.process()
            print(name, ' has been fixed...')
        else: 
            print(name, ' is not a valid parameter name. Try again... ')

        
    def process(self):   
        
        for tidx, tval in enumerate(temps):
            for midx, mval in enumerate(masses): 
                for pidx, pval in enumerate(self.parameters):               
                    
                    start_params = self.final_fishers_Full_CMBS4[tidx][midx].columns
                    if (pval.fixed==True) and (pval.name in start_params): 
                        
                        # Process Fishers
                        if (pval.name in self.final_fishers_lss[tidx][midx].columns): 
                            lss_drop_index = self.final_fishers_lss[tidx][midx].columns.get_loc(pval.name)
                            self.final_fishers_lss[tidx][midx].drop(columns=self.final_fishers_lss[tidx][midx].columns[lss_drop_index], inplace=True)
                            self.final_fishers_lss[tidx][midx].drop(index=[lss_drop_index], inplace=True)
                            self.final_fishers_lss[tidx][midx].reset_index(drop=True, inplace=True)

                        if (pval.name in self.final_fishers_Planck[tidx][midx].columns):  
                            cmb_drop_index = self.final_fishers_Planck[tidx][midx].columns.get_loc(pval.name)
                            self.final_fishers_Planck[tidx][midx].drop(columns=self.final_fishers_Planck[tidx][midx].columns[cmb_drop_index], inplace=True)
                            self.final_fishers_Planck[tidx][midx].drop(index=[cmb_drop_index], inplace=True)
                            self.final_fishers_Planck[tidx][midx].reset_index(drop=True, inplace=True)
                            
                            self.final_fishers_CMBS4[tidx][midx].drop(columns=self.final_fishers_CMBS4[tidx][midx].columns[cmb_drop_index], inplace=True)
                            self.final_fishers_CMBS4[tidx][midx].drop(index=[cmb_drop_index], inplace=True)
                            self.final_fishers_CMBS4[tidx][midx].reset_index(drop=True, inplace=True)
                            
                        full_drop_index = self.final_fishers_Full_Planck[tidx][midx].columns.get_loc(pval.name)
                        self.final_fishers_Full_Planck[tidx][midx].drop(columns=self.final_fishers_Full_Planck[tidx][midx].columns[full_drop_index], inplace=True)
                        self.final_fishers_Full_Planck[tidx][midx].drop(index=[full_drop_index], inplace=True)
                        self.final_fishers_Full_Planck[tidx][midx].reset_index(drop=True, inplace=True)
                            
                        self.final_fishers_Full_CMBS4[tidx][midx].drop(columns=self.final_fishers_Full_CMBS4[tidx][midx].columns[full_drop_index], inplace=True)
                        self.final_fishers_Full_CMBS4[tidx][midx].drop(index=[full_drop_index], inplace=True)
                        self.final_fishers_Full_CMBS4[tidx][midx].reset_index(drop=True, inplace=True)
                        
                        
                        # Process Covariances  
                        self.final_covs_lss[tidx][midx]=pd.DataFrame(np.linalg.inv(self.final_fishers_lss[tidx][midx].values), columns=self.final_fishers_lss[tidx][midx].columns) 
                        
                        self.final_covs_Planck[tidx][midx]=pd.DataFrame(np.linalg.inv(self.final_fishers_Planck[tidx][midx].values), columns=self.final_fishers_Planck[tidx][midx].columns)
                        
                        self.final_covs_CMBS4[tidx][midx]=pd.DataFrame(np.linalg.inv(self.final_fishers_CMBS4[tidx][midx].values), columns=self.final_fishers_CMBS4[tidx][midx].columns)
                        
                        self.final_covs_Full_Planck[tidx][midx]=pd.DataFrame(np.linalg.inv(self.final_fishers_Full_Planck[tidx][midx].values), columns=self.final_fishers_Full_Planck[tidx][midx].columns)
                        
                        self.final_covs_Full_CMBS4[tidx][midx]=pd.DataFrame(np.linalg.inv(self.final_fishers_Full_CMBS4[tidx][midx].values), columns=self.final_fishers_Full_CMBS4[tidx][midx].columns)
                        
                                                         
                                                         
    def print_fixed(self): 
        print('The following parameters are currently fixed: ')
        for pidx, pval in enumerate(self.parameters): 
            if pval.fixed==True: 
                print(pval.name, '\n')
        print('Execute `forecast.clear()` to marginalize... ')
                                             

## Validate that DESI-As-Euclid Matches Euclid

In [ ]:
import shutil
import seaborn as sns
import cosmicfish as cf 
import matplotlib.pyplot as plt 

# Instruct pyplot to use seaborn 
sns.set()

masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.79, 0.91, 0.94, 1.08])
param = 'N_ncdm'

boss_forecast = forecast('BOSS', '/Volumes/SSD01/TEST_RESULTS_1/BOSS/Fisher_Matrices/', masses, temps)
print('BOSS Loaded...')

desi_forecast = forecast('DESI', '/Volumes/SSD01/TEST_RESULTS_1/DESI/Fisher_Matrices/', masses, temps)
print('DESI Loaded...')

euclid_forecast = forecast('EUCLID', '/Volumes/SSD01/TEST_RESULTS_1/EUCLID/Fisher_Matrices/', masses, temps)
print('Euclid Loaded...')


boss_forecast.clear()
desi_forecast.clear()
euclid_forecast.clear()

boss_forecast.fix_param('tau_reio')
desi_forecast.fix_param('tau_reio')
euclid_forecast.fix_param('tau_reio')

boss_forecast.fix_param('beta1')
euclid_forecast.fix_param('beta1')

p1 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p2 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p3 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p4 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p5 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p1[tidx][midx]=(desi_forecast.final_fishers_lss[tidx][midx].values-euclid_forecast.final_fishers_lss[tidx][midx].values)/euclid_forecast.final_fishers_lss[tidx][midx].values
        p2[tidx][midx]=(desi_forecast.final_covs_lss[tidx][midx].values-euclid_forecast.final_covs_lss[tidx][midx].values)/euclid_forecast.final_covs_lss[tidx][midx].values
        p3[tidx][midx]=(desi_forecast.final_covs_lss[tidx][midx][param][desi_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)]-euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])/euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)]
        p4[tidx][midx]=np.sqrt(desi_forecast.final_covs_lss[tidx][midx][param][desi_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p5[tidx][midx]=np.sqrt(euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])

p1 = np.array(p1)
p2 = np.array(p2)
p3 = np.array(p3)
p4 = np.array(p4)
p5 = np.array(p5)

        
for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    for idx1 in range(len(p1[0, 0, 0, :])):
        for idx2 in range(len(p1[0, 0, 0, :])):
            plt.plot(masses, p1[tidx, :, idx1, idx2], linestyle='solid')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$((F_{ij})_{DESI}-(F_{ij})_{Euclid})/(F_{ij})_{Euclid}$", fontsize=24)
    plt.title('Fisher Relative Error, T='+str(tval)+'K', fontsize=24)
    #plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    #plt.savefig('/Users/nicholasdeporzio/Desktop/06.05.2020/F_RelErr_'+str(tval)+'K.png')
    plt.show()
    
    plt.figure(figsize=(15,7.5))
    for idx1 in range(len(p2[0, 0, 0, :])):
        for idx2 in range(len(p2[0, 0, 0, :])):
            plt.plot(masses, p2[tidx, :, idx1, idx2], linestyle='solid')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$((C_{ij})_{DESI}-(C_{ij})_{Euclid})/(C_{ij})_{Euclid}$", fontsize=24)
    plt.title('Covariance Relative Error, T='+str(tval)+'K', fontsize=24)
    #plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    #plt.savefig('/Users/nicholasdeporzio/Desktop/06.05.2020/C_RelErr_'+str(tval)+'K.png')
    plt.show()
    
    plt.figure(figsize=(15,7.5))
    plt.plot(masses, p3[tidx, :], linestyle='solid')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$((C_{g})_{DESI}-(C_{g})_{Euclid})/(C_{g})_{Euclid}$", fontsize=24)
    plt.title(r'$C(g_X)$ Relative Error, T='+str(tval)+'K', fontsize=24)
    #plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    #plt.savefig('/Users/nicholasdeporzio/Desktop/06.05.2020/Cg_RelErr_'+str(tval)+'K.png')
    plt.show()
    
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, p4[tidx, :], linestyle='solid', linewidth=4, label='DESI')
    plt.semilogx(masses, p5[tidx, :], linestyle='dashed', linewidth=4, label='EUCLID')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title(r'Error, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    #plt.savefig('/Users/nicholasdeporzio/Desktop/06.05.2020/Cg_RelErr_'+str(tval)+'K.png')
    plt.show()
    
desi_only_gErrs_tauFixed_beta1Fixed_MnuMarged = np.array(p4)
euclid_only_gErrs_tauFixed_beta1Fixed_MnuMarged = np.array(p5)

print('Beta1 fixed, M marginalized, Euclid covariances: ')
#print(desi_forecast.final_covs_lss[0][0])
print(euclid_forecast.final_covs_lss[0][0])

####################

boss_forecast.clear()
desi_forecast.clear()
euclid_forecast.clear()

boss_forecast.fix_param('tau_reio')
desi_forecast.fix_param('tau_reio')
euclid_forecast.fix_param('tau_reio')

boss_forecast.fix_param('beta1')
euclid_forecast.fix_param('beta1')

boss_forecast.fix_param('M_ncdm')
desi_forecast.fix_param('M_ncdm')
euclid_forecast.fix_param('M_ncdm')

p6 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p7 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p6[tidx][midx]=np.sqrt(desi_forecast.final_covs_lss[tidx][midx][param][desi_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p7[tidx][midx]=np.sqrt(euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])

p6 = np.array(p6)
p7 = np.array(p7)

desi_only_gErrs_tauFixed_beta1Fixed_MnuFixed = np.array(p6)
euclid_only_gErrs_tauFixed_beta1Fixed_MnuFixed = np.array(p7)

print('Beta1 fixed, M Fixed, Euclid covariances: ')
#print(desi_forecast.final_covs_lss[0][0])
print(euclid_forecast.final_covs_lss[0][0])

##############

boss_forecast.clear()
desi_forecast.clear()
euclid_forecast.clear()

boss_forecast.fix_param('tau_reio')
desi_forecast.fix_param('tau_reio')
euclid_forecast.fix_param('tau_reio')


p8 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p9 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p8[tidx][midx]=np.sqrt(desi_forecast.final_covs_lss[tidx][midx][param][desi_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p9[tidx][midx]=np.sqrt(euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])

p8 = np.array(p8)
p9 = np.array(p9)

desi_only_gErrs_tauFixed_beta1Marged_MnuMarged = np.array(p8)
euclid_only_gErrs_tauFixed_beta1Marged_MnuMarged = np.array(p9)

print('Beta1 Marginalized, M Marginalized, Euclid covariances: ')
#print(desi_forecast.final_covs_lss[0][0])
print(euclid_forecast.final_covs_lss[0][0])

##############

boss_forecast.clear()
desi_forecast.clear()
euclid_forecast.clear()

boss_forecast.fix_param('tau_reio')
desi_forecast.fix_param('tau_reio')
euclid_forecast.fix_param('tau_reio')

boss_forecast.fix_param('M_ncdm')
desi_forecast.fix_param('M_ncdm')
euclid_forecast.fix_param('M_ncdm')


p10 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p11 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p10[tidx][midx]=np.sqrt(desi_forecast.final_covs_lss[tidx][midx][param][desi_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p11[tidx][midx]=np.sqrt(euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])

p10 = np.array(p10)
p11 = np.array(p11)

desi_only_gErrs_tauFixed_beta1Marged_MnuFixed = np.array(p10)
euclid_only_gErrs_tauFixed_beta1Marged_MnuFixed = np.array(p11)

print('Beta1 Marginalized, M Fixed, Euclid covariances: ')
#print(desi_forecast.final_covs_lss[0][0])
print(euclid_forecast.final_covs_lss[0][0])


for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, euclid_only_gErrs_tauFixed_beta1Fixed_MnuMarged[tidx], linestyle='solid', color='green', label='Beta1 Fixed, M Marged')
    plt.semilogx(masses, euclid_only_gErrs_tauFixed_beta1Fixed_MnuFixed[tidx], linestyle='solid', color='red', label='Beta1 Fixed, M Fixed')
    plt.loglog(masses, euclid_only_gErrs_tauFixed_beta1Marged_MnuMarged[tidx], linestyle='dashed', linewidth=2, color='green', label='Beta1 Marged, M Marged')
    plt.loglog(masses, euclid_only_gErrs_tauFixed_beta1Marged_MnuFixed[tidx], linestyle='dashed', linewidth=2, color='red', label='Beta1 Marged, M Fixed')

    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$", fontsize=24)
    plt.title('', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    #plt.savefig('/Users/nicholasdeporzio/Desktop/06.05.2020/F_RelErr_'+str(tval)+'K.png')
    plt.show()

print(euclid_only_gErrs_tauFixed_beta1Marged_MnuMarged[0][0])
print(euclid_only_gErrs_tauFixed_beta1Marged_MnuFixed[0][0])







## Generate Paper Error Table 

In [ ]:
import shutil
import seaborn as sns
import cosmicfish as cf 
import matplotlib.pyplot as plt 

# Instruct pyplot to use seaborn 
sns.set()

masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.79, 0.91, 0.94, 1.08])


boss_forecast = forecast('BOSS', '/Volumes/SSD01/FINAL_RESULTS_NewBias/BOSS/Fisher_Matrices/', masses, temps)
print('BOSS Loaded...')
desi_forecast = forecast('DESI', '/Volumes/SSD01/FINAL_RESULTS_NewBias/DESI/Fisher_Matrices/', masses, temps)
print('DESI Loaded...')
euclid_forecast = forecast('EUCLID', '/Volumes/SSD01/FINAL_RESULTS_NewBias/EUCLID/Fisher_Matrices/', masses, temps)
print('Euclid Loaded...')

param = 'N_ncdm'

# Get LSS-Only, M_nu marginalized Errors 
p1 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p2 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p3 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

boss_forecast.fix_param('tau_reio')
desi_forecast.fix_param('tau_reio')
euclid_forecast.fix_param('tau_reio')

for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p1[tidx][midx]=np.sqrt(boss_forecast.final_covs_lss[tidx][midx][param][boss_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p2[tidx][midx]=np.sqrt(desi_forecast.final_covs_lss[tidx][midx][param][desi_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p3[tidx][midx]=np.sqrt(euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])

p1 = np.array(p1)
p2 = np.array(p2)
p3 = np.array(p3)

boss_lss_only_Mmarg = np.array(p1)
desi_lss_only_Mmarg = np.array(p2)
euclid_lss_only_Mmarg = np.array(p3)

# Get LSS-Only, M_nu fixed Errors 
p1 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p2 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p3 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

boss_forecast.fix_param('M_ncdm')
desi_forecast.fix_param('M_ncdm')
euclid_forecast.fix_param('M_ncdm')

for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p1[tidx][midx]=np.sqrt(boss_forecast.final_covs_lss[tidx][midx][param][boss_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p2[tidx][midx]=np.sqrt(desi_forecast.final_covs_lss[tidx][midx][param][desi_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])
        p3[tidx][midx]=np.sqrt(euclid_forecast.final_covs_lss[tidx][midx][param][euclid_forecast.final_covs_lss[tidx][midx].columns.get_loc(param)])

p1 = np.array(p1)
p2 = np.array(p2)
p3 = np.array(p3)

boss_lss_only_Mfixed = np.array(p1)
desi_lss_only_Mfixed = np.array(p2)
euclid_lss_only_Mfixed = np.array(p3)

# Get M_nu marginalized Errors 
p1 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p2 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p3 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p4 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p5 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p6 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p7 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p8 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p9 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p10 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p11 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p12 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

boss_forecast.clear()
desi_forecast.clear()
euclid_forecast.clear()

for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p1[tidx][midx]=np.sqrt(boss_forecast.final_covs_Planck[tidx][midx][param][boss_forecast.final_covs_Planck[tidx][midx].columns.get_loc(param)])
        p2[tidx][midx]=np.sqrt(desi_forecast.final_covs_Planck[tidx][midx][param][desi_forecast.final_covs_Planck[tidx][midx].columns.get_loc(param)])
        p3[tidx][midx]=np.sqrt(euclid_forecast.final_covs_Planck[tidx][midx][param][euclid_forecast.final_covs_Planck[tidx][midx].columns.get_loc(param)])

        p4[tidx][midx]=np.sqrt(boss_forecast.final_covs_CMBS4[tidx][midx][param][boss_forecast.final_covs_CMBS4[tidx][midx].columns.get_loc(param)])
        p5[tidx][midx]=np.sqrt(desi_forecast.final_covs_CMBS4[tidx][midx][param][desi_forecast.final_covs_CMBS4[tidx][midx].columns.get_loc(param)])
        p6[tidx][midx]=np.sqrt(euclid_forecast.final_covs_CMBS4[tidx][midx][param][euclid_forecast.final_covs_CMBS4[tidx][midx].columns.get_loc(param)])

        p7[tidx][midx]=np.sqrt(boss_forecast.final_covs_Full_Planck[tidx][midx][param][boss_forecast.final_covs_Full_Planck[tidx][midx].columns.get_loc(param)])
        p8[tidx][midx]=np.sqrt(desi_forecast.final_covs_Full_Planck[tidx][midx][param][desi_forecast.final_covs_Full_Planck[tidx][midx].columns.get_loc(param)])
        p9[tidx][midx]=np.sqrt(euclid_forecast.final_covs_Full_Planck[tidx][midx][param][euclid_forecast.final_covs_Full_Planck[tidx][midx].columns.get_loc(param)])

        p10[tidx][midx]=np.sqrt(boss_forecast.final_covs_Full_CMBS4[tidx][midx][param][boss_forecast.final_covs_Full_CMBS4[tidx][midx].columns.get_loc(param)])
        p11[tidx][midx]=np.sqrt(desi_forecast.final_covs_Full_CMBS4[tidx][midx][param][desi_forecast.final_covs_Full_CMBS4[tidx][midx].columns.get_loc(param)])
        p12[tidx][midx]=np.sqrt(euclid_forecast.final_covs_Full_CMBS4[tidx][midx][param][euclid_forecast.final_covs_Full_CMBS4[tidx][midx].columns.get_loc(param)])

p1 = np.array(p1)
p2 = np.array(p2)
p3 = np.array(p3)
p4 = np.array(p4)
p5 = np.array(p5)
p6 = np.array(p6)
p7 = np.array(p7)
p8 = np.array(p8)
p9 = np.array(p9)
p10 = np.array(p10)
p11 = np.array(p11)
p12 = np.array(p12)

boss_Planck_only_Mmarg = np.array(p1)
desi_Planck_only_Mmarg = np.array(p2)
euclid_Planck_only_Mmarg = np.array(p3)
boss_CMBS4_only_Mmarg = np.array(p4)
desi_CMBS4_only_Mmarg = np.array(p5)
euclid_CMBS4_only_Mmarg = np.array(p6)

boss_Full_Planck_Mmarg = np.array(p7)
desi_Full_Planck_Mmarg = np.array(p8)
euclid_Full_Planck_Mmarg = np.array(p9)
boss_Full_CMBS4_Mmarg = np.array(p10)
desi_Full_CMBS4_Mmarg = np.array(p11)
euclid_Full_CMBS4_Mmarg = np.array(p12)

# Get M_nu fixed Errors 
p1 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p2 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p3 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p4 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p5 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p6 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p7 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p8 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p9 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p10 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p11 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]
p12 = [[0. for midx, mval in enumerate(masses)] for tidx, tval in enumerate(temps)]

boss_forecast.fix_param('M_ncdm')
desi_forecast.fix_param('M_ncdm')
euclid_forecast.fix_param('M_ncdm')


for tidx, tval in enumerate(temps): 
    for midx, mval in enumerate(masses): 
        p1[tidx][midx]=np.sqrt(boss_forecast.final_covs_Planck[tidx][midx][param][boss_forecast.final_covs_Planck[tidx][midx].columns.get_loc(param)])
        p2[tidx][midx]=np.sqrt(desi_forecast.final_covs_Planck[tidx][midx][param][desi_forecast.final_covs_Planck[tidx][midx].columns.get_loc(param)])
        p3[tidx][midx]=np.sqrt(euclid_forecast.final_covs_Planck[tidx][midx][param][euclid_forecast.final_covs_Planck[tidx][midx].columns.get_loc(param)])

        p4[tidx][midx]=np.sqrt(boss_forecast.final_covs_CMBS4[tidx][midx][param][boss_forecast.final_covs_CMBS4[tidx][midx].columns.get_loc(param)])
        p5[tidx][midx]=np.sqrt(desi_forecast.final_covs_CMBS4[tidx][midx][param][desi_forecast.final_covs_CMBS4[tidx][midx].columns.get_loc(param)])
        p6[tidx][midx]=np.sqrt(euclid_forecast.final_covs_CMBS4[tidx][midx][param][euclid_forecast.final_covs_CMBS4[tidx][midx].columns.get_loc(param)])

        p7[tidx][midx]=np.sqrt(boss_forecast.final_covs_Full_Planck[tidx][midx][param][boss_forecast.final_covs_Full_Planck[tidx][midx].columns.get_loc(param)])
        p8[tidx][midx]=np.sqrt(desi_forecast.final_covs_Full_Planck[tidx][midx][param][desi_forecast.final_covs_Full_Planck[tidx][midx].columns.get_loc(param)])
        p9[tidx][midx]=np.sqrt(euclid_forecast.final_covs_Full_Planck[tidx][midx][param][euclid_forecast.final_covs_Full_Planck[tidx][midx].columns.get_loc(param)])

        p10[tidx][midx]=np.sqrt(boss_forecast.final_covs_Full_CMBS4[tidx][midx][param][boss_forecast.final_covs_Full_CMBS4[tidx][midx].columns.get_loc(param)])
        p11[tidx][midx]=np.sqrt(desi_forecast.final_covs_Full_CMBS4[tidx][midx][param][desi_forecast.final_covs_Full_CMBS4[tidx][midx].columns.get_loc(param)])
        p12[tidx][midx]=np.sqrt(euclid_forecast.final_covs_Full_CMBS4[tidx][midx][param][euclid_forecast.final_covs_Full_CMBS4[tidx][midx].columns.get_loc(param)])

p1 = np.array(p1)
p2 = np.array(p2)
p3 = np.array(p3)
p4 = np.array(p4)
p5 = np.array(p5)
p6 = np.array(p6)
p7 = np.array(p7)
p8 = np.array(p8)
p9 = np.array(p9)
p10 = np.array(p10)
p11 = np.array(p11)
p12 = np.array(p12)

boss_Planck_only_Mfixed = np.array(p1)
desi_Planck_only_Mfixed = np.array(p2)
euclid_Planck_only_Mfixed = np.array(p3)
boss_CMBS4_only_Mfixed = np.array(p4)
desi_CMBS4_only_Mfixed = np.array(p5)
euclid_CMBS4_only_Mfixed = np.array(p6)

boss_Full_Planck_Mfixed = np.array(p7)
desi_Full_Planck_Mfixed = np.array(p8)
euclid_Full_Planck_Mfixed = np.array(p9)
boss_Full_CMBS4_Mfixed = np.array(p10)
desi_Full_CMBS4_Mfixed = np.array(p11)
euclid_Full_CMBS4_Mfixed = np.array(p12)

savepath='/Users/nicholasdeporzio/Desktop/06.05.2020/'

np.savetxt(savepath+'boss_lss_only_Mnu_marginalized.txt', boss_lss_only_Mmarg)
np.savetxt(savepath+'boss_lss_only_Mnu_fixed.txt', boss_lss_only_Mfixed)
np.savetxt(savepath+'boss_Planck_only_Mnu_marginalized.txt', boss_Planck_only_Mmarg)
np.savetxt(savepath+'boss_Planck_only_Mnu_fixed.txt', boss_Planck_only_Mfixed)
np.savetxt(savepath+'boss_CMBS4_only_Mnu_marginalized.txt', boss_CMBS4_only_Mmarg)
np.savetxt(savepath+'boss_CMBS4_only_Mnu_fixed.txt', boss_CMBS4_only_Mfixed)
np.savetxt(savepath+'boss_Full_Planck_Mnu_marginalized.txt', boss_Full_Planck_Mmarg)
np.savetxt(savepath+'boss_Full_Planck_Mnu_fixed.txt', boss_Full_Planck_Mfixed)
np.savetxt(savepath+'boss_Full_CMBS4_Mnu_marginalized.txt', boss_Full_CMBS4_Mmarg)
np.savetxt(savepath+'boss_Full_CMBS4_Mnu_fixed.txt', boss_Full_CMBS4_Mfixed)

np.savetxt(savepath+'desi_lss_only_Mnu_marginalized.txt', desi_lss_only_Mmarg)
np.savetxt(savepath+'desi_lss_only_Mnu_fixed.txt', desi_lss_only_Mfixed)
np.savetxt(savepath+'desi_Planck_only_Mnu_marginalized.txt', desi_Planck_only_Mmarg)
np.savetxt(savepath+'desi_Planck_only_Mnu_fixed.txt', desi_Planck_only_Mfixed)
np.savetxt(savepath+'desi_CMBS4_only_Mnu_marginalized.txt', desi_CMBS4_only_Mmarg)
np.savetxt(savepath+'desi_CMBS4_only_Mnu_fixed.txt', desi_CMBS4_only_Mfixed)
np.savetxt(savepath+'desi_Full_Planck_Mnu_marginalized.txt', desi_Full_Planck_Mmarg)
np.savetxt(savepath+'desi_Full_Planck_Mnu_fixed.txt', desi_Full_Planck_Mfixed)
np.savetxt(savepath+'desi_Full_CMBS4_Mnu_marginalized.txt', desi_Full_CMBS4_Mmarg)
np.savetxt(savepath+'desi_Full_CMBS4_Mnu_fixed.txt', desi_Full_CMBS4_Mfixed)

np.savetxt(savepath+'euclid_lss_only_Mnu_marginalized.txt', euclid_lss_only_Mmarg)
np.savetxt(savepath+'euclid_lss_only_Mnu_fixed.txt', euclid_lss_only_Mfixed)
np.savetxt(savepath+'euclid_Planck_only_Mnu_marginalized.txt', euclid_Planck_only_Mmarg)
np.savetxt(savepath+'euclid_Planck_only_Mnu_fixed.txt', euclid_Planck_only_Mfixed)
np.savetxt(savepath+'euclid_CMBS4_only_Mnu_marginalized.txt', euclid_CMBS4_only_Mmarg)
np.savetxt(savepath+'euclid_CMBS4_only_Mnu_fixed.txt', euclid_CMBS4_only_Mfixed)
np.savetxt(savepath+'euclid_Full_Planck_Mnu_marginalized.txt', euclid_Full_Planck_Mmarg)
np.savetxt(savepath+'euclid_Full_Planck_Mnu_fixed.txt', euclid_Full_Planck_Mfixed)
np.savetxt(savepath+'euclid_Full_CMBS4_Mnu_marginalized.txt', euclid_Full_CMBS4_Mmarg)
np.savetxt(savepath+'euclid_Full_CMBS4_Mnu_fixed.txt', euclid_Full_CMBS4_Mfixed)
        
for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, boss_lss_only_Mmarg[tidx, :], linestyle='solid', color='blue', label='BOSS, M Marginalized')
    plt.semilogx(masses, boss_lss_only_Mfixed[tidx, :], linestyle='dashed', color='blue', label='BOSS, M Fixed')
    plt.semilogx(masses, desi_lss_only_Mmarg[tidx, :], linestyle='solid', color='orange', label='DESI, M Marginalized')
    plt.semilogx(masses, desi_lss_only_Mfixed[tidx, :], linestyle='dashed', color='orange', label='DESI, M Fixed')
    plt.semilogx(masses, euclid_lss_only_Mmarg[tidx, :], linestyle='solid', color='pink', label='Euclid, M Marginalized')
    plt.semilogx(masses, euclid_lss_only_Mfixed[tidx, :], linestyle='dashed', color='pink', label='Euclid, M Fixed')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title('LSS Only, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    plt.savefig(savepath+'lss_only_'+str(tval)+'K.png')
    plt.show()
    
for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, boss_Planck_only_Mmarg[tidx, :], linestyle='solid', color='blue', label='Planck, M Marginalized')
    plt.semilogx(masses, boss_Planck_only_Mfixed[tidx, :], linestyle='dashed', color='blue', label='Planck, M Fixed')
    plt.semilogx(masses, boss_CMBS4_only_Mmarg[tidx, :], linestyle='solid', color='orange', label='CMBS4, M Marginalized')
    plt.semilogx(masses, boss_CMBS4_only_Mfixed[tidx, :], linestyle='dashed', color='orange', label='CMBS4, M Fixed')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title('CMB Only, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    plt.savefig(savepath+'cmb_only_'+str(tval)+'K.png')
    plt.show()
    
for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, boss_Full_Planck_Mmarg[tidx, :], linestyle='solid', color='blue', label='BOSS, M Marginalized')
    plt.semilogx(masses, boss_Full_Planck_Mfixed[tidx, :], linestyle='dashed', color='blue', label='BOSS, M Fixed')
    plt.semilogx(masses, desi_Full_Planck_Mmarg[tidx, :], linestyle='solid', color='orange', label='DESI, M Marginalized')
    plt.semilogx(masses, desi_Full_Planck_Mfixed[tidx, :], linestyle='dashed', color='orange', label='DESI, M Fixed')
    plt.semilogx(masses, euclid_Full_Planck_Mmarg[tidx, :], linestyle='solid', color='pink', label='Euclid, M Marginalized')
    plt.semilogx(masses, euclid_Full_Planck_Mfixed[tidx, :], linestyle='dashed', color='pink', label='Euclid, M Fixed')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title('LSS + Planck, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    plt.savefig(savepath+'Full_Planck_'+str(tval)+'K.png')
    plt.show()
    
for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, boss_Full_CMBS4_Mmarg[tidx, :], linestyle='solid', color='blue', label='BOSS, M Marginalized')
    plt.semilogx(masses, boss_Full_CMBS4_Mfixed[tidx, :], linestyle='dashed', color='blue', label='BOSS, M Fixed')
    plt.semilogx(masses, desi_Full_CMBS4_Mmarg[tidx, :], linestyle='solid', color='orange', label='DESI, M Marginalized')
    plt.semilogx(masses, desi_Full_CMBS4_Mfixed[tidx, :], linestyle='dashed', color='orange', label='DESI, M Fixed')
    plt.semilogx(masses, euclid_Full_CMBS4_Mmarg[tidx, :], linestyle='solid', color='pink', label='Euclid, M Marginalized')
    plt.semilogx(masses, euclid_Full_CMBS4_Mfixed[tidx, :], linestyle='dashed', color='pink', label='Euclid, M Fixed')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title('LSS + CMBS4, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    plt.savefig(savepath+'Full_CMBS4_'+str(tval)+'K.png')
    plt.show()

for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, boss_lss_only_Mmarg[tidx, :], linestyle='solid', color='blue', label='BOSS Only, M Marginalized')
    plt.semilogx(masses, boss_lss_only_Mfixed[tidx, :], linestyle='dashed', color='blue', label='BOSS Only, M Fixed')
    plt.semilogx(masses, boss_Full_Planck_Mmarg[tidx, :], linestyle='solid', color='orange', label='BOSS+Planck, M Marginalized')
    plt.semilogx(masses, boss_Full_Planck_Mfixed[tidx, :], linestyle='dashed', color='orange', label='BOSS+Planck, M Fixed')
    plt.semilogx(masses, boss_Full_CMBS4_Mmarg[tidx, :], linestyle='solid', color='pink', label='BOSS+CMBS4, M Marginalized')
    plt.semilogx(masses, boss_Full_CMBS4_Mfixed[tidx, :], linestyle='dashed', color='pink', label='BOSS+CMBS4, M Fixed')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title('LSS + CMBS4, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    plt.savefig(savepath+'boss_all_'+str(tval)+'K.png')
    plt.show()
    
for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, desi_lss_only_Mmarg[tidx, :], linestyle='solid', color='blue', label='DESI Only, M Marginalized')
    plt.semilogx(masses, desi_lss_only_Mfixed[tidx, :], linestyle='dashed', color='blue', label='DESI Only, M Fixed')
    plt.semilogx(masses, desi_Full_Planck_Mmarg[tidx, :], linestyle='solid', color='orange', label='DESI+Planck, M Marginalized')
    plt.semilogx(masses, desi_Full_Planck_Mfixed[tidx, :], linestyle='dashed', color='orange', label='DESI+Planck, M Fixed')
    plt.semilogx(masses, desi_Full_CMBS4_Mmarg[tidx, :], linestyle='solid', color='pink', label='DESI+CMBS4, M Marginalized')
    plt.semilogx(masses, desi_Full_CMBS4_Mfixed[tidx, :], linestyle='dashed', color='pink', label='DESI+CMBS4, M Fixed')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title('LSS + CMBS4, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    plt.savefig(savepath+'desi_all_'+str(tval)+'K.png')
    plt.show()
    
for tidx, tval in enumerate(temps): 
    plt.figure(figsize=(15,7.5))
    plt.semilogx(masses, euclid_lss_only_Mmarg[tidx, :], linestyle='solid', color='blue', label='Euclid Only, M Marginalized')
    plt.semilogx(masses, euclid_lss_only_Mfixed[tidx, :], linestyle='dashed', color='blue', label='Euclid Only, M Fixed')
    plt.semilogx(masses, euclid_Full_Planck_Mmarg[tidx, :], linestyle='solid', color='orange', label='Euclid+Planck, M Marginalized')
    plt.semilogx(masses, euclid_Full_Planck_Mfixed[tidx, :], linestyle='dashed', color='orange', label='Euclid+Planck, M Fixed')
    plt.semilogx(masses, euclid_Full_CMBS4_Mmarg[tidx, :], linestyle='solid', color='pink', label='Euclid+CMBS4, M Marginalized')
    plt.semilogx(masses, euclid_Full_CMBS4_Mfixed[tidx, :], linestyle='dashed', color='pink', label='Euclid+CMBS4, M Fixed')
    plt.xlabel(r"Relic Mass [eV]", fontsize=24)                                       
    plt.ylabel(r"$\sigma(g_X)$", fontsize=24)
    plt.title('LSS + CMBS4, T='+str(tval)+'K', fontsize=24)
    plt.legend(fontsize=18, loc='upper right')                                       
    plt.tick_params(axis='x', which='minttor')                                      
    plt.grid(True, which='minor')                 
    #plt.xlim(1, 5)
    #plt.ylim(0, 1.5)
    plt.savefig(savepath+'euclid_all_'+str(tval)+'K.png')
    plt.show()

## Interpolate Error at 3-sigma

In [ ]:
import scipy.interpolate
import numpy as np

importpath = '/Users/nicholasdeporzio/Desktop/Final_Results_Hold_06.05.2020/N_errors/'

boss_lss_only_Mnu_marginalized = np.loadtxt(importpath+'boss_lss_only_Mnu_marginalized.txt')
boss_lss_only_Mnu_fixed = np.loadtxt(importpath+'boss_lss_only_Mnu_fixed.txt')
boss_Planck_only_Mnu_marginalized = np.loadtxt(importpath+'boss_Planck_only_Mnu_marginalized.txt')
boss_Planck_only_Mnu_fixed = np.loadtxt(importpath+'boss_Planck_only_Mnu_fixed.txt')
boss_CMBS4_only_Mnu_marginalized = np.loadtxt(importpath+'boss_CMBS4_only_Mnu_marginalized.txt')
boss_CMBS4_only_Mnu_fixed = np.loadtxt(importpath+'boss_CMBS4_only_Mnu_fixed.txt')
boss_Full_Planck_Mnu_marginalized = np.loadtxt(importpath+'boss_Full_Planck_Mnu_marginalized.txt')
boss_Full_Planck_Mnu_fixed = np.loadtxt(importpath+'boss_Full_Planck_Mnu_fixed.txt')
boss_Full_CMBS4_Mnu_marginalized = np.loadtxt(importpath+'boss_Full_CMBS4_Mnu_marginalized.txt')
boss_Full_CMBS4_Mnu_fixed = np.loadtxt(importpath+'boss_Full_CMBS4_Mnu_fixed.txt')

desi_lss_only_Mnu_marginalized = np.loadtxt(importpath+'desi_lss_only_Mnu_marginalized.txt')
desi_lss_only_Mnu_fixed = np.loadtxt(importpath+'desi_lss_only_Mnu_fixed.txt')
desi_Planck_only_Mnu_marginalized = np.loadtxt(importpath+'desi_Planck_only_Mnu_marginalized.txt')
desi_Planck_only_Mnu_fixed = np.loadtxt(importpath+'desi_Planck_only_Mnu_fixed.txt')
desi_CMBS4_only_Mnu_marginalized = np.loadtxt(importpath+'desi_CMBS4_only_Mnu_marginalized.txt')
desi_CMBS4_only_Mnu_fixed = np.loadtxt(importpath+'desi_CMBS4_only_Mnu_fixed.txt')
desi_Full_Planck_Mnu_marginalized = np.loadtxt(importpath+'desi_Full_Planck_Mnu_marginalized.txt')
desi_Full_Planck_Mnu_fixed = np.loadtxt(importpath+'desi_Full_Planck_Mnu_fixed.txt')
desi_Full_CMBS4_Mnu_marginalized = np.loadtxt(importpath+'desi_Full_CMBS4_Mnu_marginalized.txt')
desi_Full_CMBS4_Mnu_fixed = np.loadtxt(importpath+'desi_Full_CMBS4_Mnu_fixed.txt')

euclid_lss_only_Mnu_marginalized = np.loadtxt(importpath+'euclid_lss_only_Mnu_marginalized.txt')
euclid_lss_only_Mnu_fixed = np.loadtxt(importpath+'euclid_lss_only_Mnu_fixed.txt')
euclid_Planck_only_Mnu_marginalized = np.loadtxt(importpath+'euclid_Planck_only_Mnu_marginalized.txt')
euclid_Planck_only_Mnu_fixed = np.loadtxt(importpath+'euclid_Planck_only_Mnu_fixed.txt')
euclid_CMBS4_only_Mnu_marginalized = np.loadtxt(importpath+'euclid_CMBS4_only_Mnu_marginalized.txt')
euclid_CMBS4_only_Mnu_fixed = np.loadtxt(importpath+'euclid_CMBS4_only_Mnu_fixed.txt')
euclid_Full_Planck_Mnu_marginalized = np.loadtxt(importpath+'euclid_Full_Planck_Mnu_marginalized.txt')
euclid_Full_Planck_Mnu_fixed = np.loadtxt(importpath+'euclid_Full_Planck_Mnu_fixed.txt')
euclid_Full_CMBS4_Mnu_marginalized = np.loadtxt(importpath+'euclid_Full_CMBS4_Mnu_marginalized.txt')
euclid_Full_CMBS4_Mnu_fixed = np.loadtxt(importpath+'euclid_Full_CMBS4_Mnu_fixed.txt')

temp_index=0
masses = np.geomspace(0.01, 10., 21)
temps=[0.79, 0.91, 0.94, 1.08] 
surveys=['boss', 'desi', 'euclid']

if temp_index==0: 
    gamma=1
    gX=1
elif temp_index==1:
    gamma=0
    gX=2
elif temp_index==2: 
    gamma=1
    gX=2
elif temp_index==3: 
    gamma=0
    gX=4

#sigma_threshold = gX/3.
sigma_threshold = 1./3.

def m_convert(m_eq): 
    return m_eq * np.power(2./gX, 1./4.) * np.power(7./8., gamma/4.) * np.power(6./7., gamma)

print('Temperature: ', temps[temp_index])

int_boss_lss_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_lss_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_boss_lss_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_lss_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_boss_Planck_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_Planck_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_boss_Planck_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_Planck_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_boss_CMBS4_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_CMBS4_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_boss_CMBS4_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_CMBS4_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_boss_Full_Planck_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_Full_Planck_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_boss_Full_Planck_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_Full_Planck_Mnu_fixed[temp_index, :]-sigma_threshold)
int_boss_Full_CMBS4_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_Full_CMBS4_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_boss_Full_CMBS4_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, boss_Full_CMBS4_Mnu_fixed[temp_index, :]-sigma_threshold)

int_desi_lss_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_lss_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_desi_lss_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_lss_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_desi_Planck_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_Planck_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_desi_Planck_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_Planck_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_desi_CMBS4_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_CMBS4_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_desi_CMBS4_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_CMBS4_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_desi_Full_Planck_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_Full_Planck_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_desi_Full_Planck_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_Full_Planck_Mnu_fixed[temp_index, :]-sigma_threshold)
int_desi_Full_CMBS4_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_Full_CMBS4_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_desi_Full_CMBS4_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, desi_Full_CMBS4_Mnu_fixed[temp_index, :]-sigma_threshold)

int_euclid_lss_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_lss_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_euclid_lss_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_lss_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_euclid_Planck_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_Planck_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_euclid_Planck_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_Planck_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_euclid_CMBS4_only_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_CMBS4_only_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_euclid_CMBS4_only_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_CMBS4_only_Mnu_fixed[temp_index, :]-sigma_threshold)
int_euclid_Full_Planck_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_Full_Planck_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_euclid_Full_Planck_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_Full_Planck_Mnu_fixed[temp_index, :]-sigma_threshold)
int_euclid_Full_CMBS4_Mnu_marginalized = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_Full_CMBS4_Mnu_marginalized[temp_index, :]-sigma_threshold)
int_euclid_Full_CMBS4_Mnu_fixed = scipy.interpolate.InterpolatedUnivariateSpline(masses, euclid_Full_CMBS4_Mnu_fixed[temp_index, :]-sigma_threshold)

print('**************************')
print('BOSS Only (M_nu Marginalized): ', m_convert(int_boss_lss_only_Mnu_marginalized.roots()))
print('BOSS Only (M_nu Fixed): ', m_convert(int_boss_lss_only_Mnu_fixed.roots()))
print('Planck Only (M_nu Marginalized): ', m_convert(int_boss_Planck_only_Mnu_marginalized.roots()))
print('Planck Only (M_nu Fixed): ', m_convert(int_boss_Planck_only_Mnu_fixed.roots()))
print('CMBS4 Only (M_nu Marginalized): ', m_convert(int_boss_CMBS4_only_Mnu_marginalized.roots()))
print('CMBS4 Only (M_nu Fixed): ', m_convert(int_boss_CMBS4_only_Mnu_fixed.roots()))
print('BOSS + Planck (M_nu Marginalized): ', m_convert(int_boss_Full_Planck_Mnu_marginalized.roots()))
print('BOSS + Planck (M_nu Fixed): ', m_convert(int_boss_Full_Planck_Mnu_fixed.roots()))
print('BOSS + CMBS4 (M_nu Marginalized): ', m_convert(int_boss_Full_CMBS4_Mnu_marginalized.roots()))
print('BOSS + CMBS4 (M_nu Fixed): ', m_convert(int_boss_Full_CMBS4_Mnu_fixed.roots()))
print('**************************')
print('DESI Only (M_nu Marginalized): ', m_convert(int_desi_lss_only_Mnu_marginalized.roots()))
print('DESI Only (M_nu Fixed): ', m_convert(int_desi_lss_only_Mnu_fixed.roots()))
print('Planck Only (M_nu Marginalized): ', m_convert(int_desi_Planck_only_Mnu_marginalized.roots()))
print('Planck Only (M_nu Fixed): ', m_convert(int_desi_Planck_only_Mnu_fixed.roots()))
print('CMBS4 Only (M_nu Marginalized): ', m_convert(int_desi_CMBS4_only_Mnu_marginalized.roots()))
print('CMBS4 Only (M_nu Fixed): ', m_convert(int_desi_CMBS4_only_Mnu_fixed.roots()))
print('DESI + Planck (M_nu Marginalized): ', m_convert(int_desi_Full_Planck_Mnu_marginalized.roots()))
print('DESI + Planck (M_nu Fixed): ', m_convert(int_desi_Full_Planck_Mnu_fixed.roots()))
print('DESI + CMBS4 (M_nu Marginalized): ', m_convert(int_desi_Full_CMBS4_Mnu_marginalized.roots()))
print('DESI + CMBS4 (M_nu Fixed): ', m_convert(int_desi_Full_CMBS4_Mnu_fixed.roots()))
print('**************************')
print('Euclid Only (M_nu Marginalized): ', m_convert(int_euclid_lss_only_Mnu_marginalized.roots()))
print('Euclid Only (M_nu Fixed): ', m_convert(int_euclid_lss_only_Mnu_fixed.roots()))
print('Planck Only (M_nu Marginalized): ', m_convert(int_euclid_Planck_only_Mnu_marginalized.roots()))
print('Planck Only (M_nu Fixed): ', m_convert(int_euclid_Planck_only_Mnu_fixed.roots()))
print('CMBS4 Only (M_nu Marginalized): ', m_convert(int_euclid_CMBS4_only_Mnu_marginalized.roots()))
print('CMBS4 Only (M_nu Fixed): ', m_convert(int_euclid_CMBS4_only_Mnu_fixed.roots()))
print('Euclid + Planck (M_nu Marginalized): ', m_convert(int_euclid_Full_Planck_Mnu_marginalized.roots()))
print('Euclid + Planck (M_nu Fixed): ', m_convert(int_euclid_Full_Planck_Mnu_fixed.roots()))
print('Euclid + CMBS4 (M_nu Marginalized): ', m_convert(int_euclid_Full_CMBS4_Mnu_marginalized.roots()))
print('Euclid + CMBS4 (M_nu Fixed): ', m_convert(int_euclid_Full_CMBS4_Mnu_fixed.roots()))
print('**************************')



## Equivalent N_eff

In [ ]:
def sig_Neff(g_x, sig_g, T_x): 
    return g_x * (sig_g/2.) * np.power(T_x/1.95, 4)

In [ ]:
for tidx, tval in enumerate(temps): 
    if tidx==0: 
        gx = 1.
    elif tidx==1: 
        gx = 2
    elif tidx==2: 
        gx = 2
    elif tidx==3: 
        gx = 3
        
    print('Error on N_eff for T=', tval, '...')
    print('BOSS Only: ', sig_Neff(gx, boss_lss_only_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, boss_lss_only_Mnu_fixed[tidx][0], tval), ')')
    print('DESI Only: ', sig_Neff(gx, desi_lss_only_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, desi_lss_only_Mnu_fixed[tidx][0], tval), ')')
    print('Euclid Only: ', sig_Neff(gx, euclid_lss_only_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, euclid_lss_only_Mnu_fixed[tidx][0], tval), ')')

    print('Planck Only: ', sig_Neff(gx, desi_Planck_only_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, boss_Planck_only_Mnu_fixed[tidx][0], tval), ')')
    print('CMBS4 Only: ', sig_Neff(gx, desi_CMBS4_only_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, boss_CMBS4_only_Mnu_fixed[tidx][0], tval), ')')

    print('BOSS + Planck: ', sig_Neff(gx, boss_Full_Planck_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, boss_Full_Planck_Mnu_fixed[tidx][0], tval), ')')
    print('DESI + Planck: ', sig_Neff(gx, desi_Full_Planck_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, desi_Full_Planck_Mnu_fixed[tidx][0], tval), ')')
    print('Euclid + Planck: ', sig_Neff(gx, euclid_Full_Planck_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, euclid_Full_Planck_Mnu_fixed[tidx][0], tval), ')')

    print('BOSS + CMBS4: ', sig_Neff(gx, boss_Full_CMBS4_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, boss_Full_CMBS4_Mnu_fixed[tidx][0], tval), ')')
    print('DESI + CMBS4: ', sig_Neff(gx, desi_Full_CMBS4_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, desi_Full_CMBS4_Mnu_fixed[tidx][0], tval), ')')
    print('Euclid + CMBS4: ', sig_Neff(gx, euclid_Full_CMBS4_Mnu_marginalized[tidx][0], tval), ' (', sig_Neff(gx, euclid_Full_CMBS4_Mnu_fixed[tidx][0], tval), ')')
    print('********************')

In [ ]:
for tidx, tval in enumerate(temps): 
    if tidx==0: 
        gx = 1.
    elif tidx==1: 
        gx = 2
    elif tidx==2: 
        gx = 2
    elif tidx==3: 
        gx = 3
        
    print('Marginalization Shift in Error for T=', tval, '...')
    print('BOSS Only: ', max((boss_lss_only_Mnu_marginalized[tidx]/boss_lss_only_Mnu_fixed[tidx])))
    print('DESI Only: ', max(desi_lss_only_Mnu_marginalized[tidx]/desi_lss_only_Mnu_fixed[tidx]))
    print('Euclid Only: ', max(euclid_lss_only_Mnu_marginalized[tidx]/euclid_lss_only_Mnu_fixed[tidx]))

    print('Planck Only: ', max(desi_Planck_only_Mnu_marginalized[tidx]/desi_Planck_only_Mnu_fixed[tidx]))
    print('CMBS4 Only: ', max(desi_CMBS4_only_Mnu_marginalized[tidx]/desi_CMBS4_only_Mnu_fixed[tidx]))

    print('BOSS + Planck: ', max(boss_Full_Planck_Mnu_marginalized[tidx]/boss_Full_Planck_Mnu_fixed[tidx]))
    print('DESI + Planck: ', max(desi_Full_Planck_Mnu_marginalized[tidx]/desi_Full_Planck_Mnu_fixed[tidx]))
    print('Euclid + Planck: ', max(euclid_Full_Planck_Mnu_marginalized[tidx]/euclid_Full_Planck_Mnu_fixed[tidx]))

    print('BOSS + CMBS4: ', max(boss_Full_CMBS4_Mnu_marginalized[tidx]/boss_Full_CMBS4_Mnu_fixed[tidx]))
    print('DESI + CMBS4: ', max(desi_Full_CMBS4_Mnu_marginalized[tidx]/desi_Full_CMBS4_Mnu_fixed[tidx]))
    print('Euclid + CMBS4: ', max(euclid_Full_CMBS4_Mnu_marginalized[tidx]/euclid_Full_CMBS4_Mnu_fixed[tidx]))
    print('********************')

# Debugging: Plot Contours

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

#dataset = 'Full'
#dataset = 'LSS'

#survey = 'CMBS4'
survey = 'Planck'

dtype='cmb'
mass_index=13
temp_index=0

masses = np.geomspace(0.01, 10., 21)
temps = np.array([0.79, 0.91, 0.94, 1.08])
mass=masses[mass_index]
temp=temps[temp_index]
path='/Volumes/SSD01/FINAL_RESULTS/Fine_Plot/Fisher_Matrices/T'+str(temp)+'K/M'+f'{mass:.3f}'+'/'+survey+'/inv_'+dtype+'fisher.mat'

cov=np.loadtxt(path, skiprows=1)

fid = [
    2.2321e-9, #A_s
    0.701, #h
    0.967, #n_s
    0.02226, #omega_b
    0.1127, #omega_cdm
    0.0598, #tau_reio
    0.06, #M_ncdm
    1.0, #N_ncdm
#   250., #sigma_fog
#    1.0, #beta0
#    1.0, #alphak2 
]

params = [
    r'$A_s$', 
    r'$h$', 
    r'$n_s$', 
    r'$\omega_b$', 
    r'$\omega_{cdm}$', 
    r'$\tau_{reio}$', 
    r'$M_\nu$',
    r'$g_\chi$',  
#    r'$\sigma_{fog}$', 
#    r'$\beta_0$', 
#    r'$\alpha_{k2}$'
]

gauss = GaussianND(fid, cov, names=params)

h = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
h.settings.figure_legend_frame = False #Box around legend
h.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
h.settings.axes_fontsize=14
h.settings.axes_labelsize=20
h.settings.axis_tick_x_rotation=45
h.settings.legend_fontsize=20
#h.settings.axis_tick_y_rotation=90
#h.settings.colorbar_tick_rotation=90


h.triangle_plot(
    [
        gauss
    ], 
    filled=True,
    legend_labels=[
        "Planck, T=0.79, M="+f'{mass:.3f}'
    ], 
    line_args=[
        {'ls':'solid', 'lw':2, 'color':'#A51C30'},
    ], 
    contour_args=[
        {'ls':'dotted', 'lw':2, 'color':'#A51C30', 'filled' : True},
    ])
plt.savefig("/Users/nicholasdeporzio/Desktop/test/"+dtype+"_M"+f'{mass:.3f}'+".png")



In [ ]:
np.geomspace(0.01, 10., 21)[6]